# 🔧 Preprocessing Steps
🧹 Data Cleaning:
- Handled missing values and checked data consistency before further processing.

🧬 Categorical Encoding:
- Transformed categorical variables into numeric using Label Encoding and One-Hot Encoding depending on the nature of the feature.

📊 Feature Separation:
- Identified and separated categorical and numerical features to apply appropriate transformations.

📏 Feature Scaling:
- Applied StandardScaler to normalize numerical features for faster and more stable model training.

🎯 Target Preparation:
- Converted the target variable into binary format suitable for binary classification with neural networks.

In [1]:
import pandas as pd
import numpy as np

import optuna

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD, Adam, RMSprop, Adagrad, Adadelta, Nadam

from sklearn.metrics import roc_auc_score

In [2]:
data = pd.read_csv(r'Employee.csv')
data

,Education,JoiningYear,City,PaymentTier,Age,Gender,EverBenched,ExperienceInCurrentDomain,LeaveOrNot
0,Bachelors,2017,Bangalore,3,34,Male,No,0,0
1,Bachelors,2013,Pune,1,28,Female,No,3,1
2,Bachelors,2014,New Delhi,3,38,Female,No,2,0
3,Masters,2016,Bangalore,3,27,Male,No,5,1
4,Masters,2017,Pune,3,24,Male,Yes,2,1
...,...,...,...,...,...,...,...,...,...
4648,Bachelors,2013,Bangalore,3,26,Female,No,4,0
4649,Masters,2013,Pune,2,37,Male,No,2,1
4650,Masters,2018,New Delhi,3,27,Male,No,5,1
4651,Bachelors,2012,Bangalore,3,30,Male,Yes,2,0


In [3]:
data.describe(include = 'all')

,Education,JoiningYear,City,PaymentTier,Age,Gender,EverBenched,ExperienceInCurrentDomain,LeaveOrNot
count,4653,4653.000000,4653,4653.000000,4653.000000,4653,4653,4653.000000,4653.000000
unique,3,NaN,3,NaN,NaN,2,2,NaN,NaN
top,Bachelors,NaN,Bangalore,NaN,NaN,Male,No,NaN,NaN
freq,3601,NaN,2228,NaN,NaN,2778,4175,NaN,NaN
mean,NaN,2015.062970,NaN,2.698259,29.393295,NaN,NaN,2.905652,0.343864
std,NaN,1.863377,NaN,0.561435,4.826087,NaN,NaN,1.558240,0.475047
min,NaN,2012.000000,NaN,1.000000,22.000000,NaN,NaN,0.000000,0.000000
25%,NaN,2013.000000,NaN,3.000000,26.000000,NaN,NaN,2.000000,0.000000
50%,NaN,2015.000000,NaN,3.000000,28.000000,NaN,NaN,3.000000,0.000000
75%,NaN,2017.000000,NaN,3.000000,32.000000,NaN,NaN,4.000000,1.000000


In [4]:
data = pd.get_dummies(data, drop_first = True,dtype = int)

data

,JoiningYear,PaymentTier,Age,ExperienceInCurrentDomain,LeaveOrNot,Education_Masters,Education_PHD,City_New Delhi,City_Pune,Gender_Male,EverBenched_Yes
0,2017,3,34,0,0,0,0,0,0,1,0
1,2013,1,28,3,1,0,0,0,1,0,0
2,2014,3,38,2,0,0,0,1,0,0,0
3,2016,3,27,5,1,1,0,0,0,1,0
4,2017,3,24,2,1,1,0,0,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...
4648,2013,3,26,4,0,0,0,0,0,0,0
4649,2013,2,37,2,1,1,0,0,1,1,0
4650,2018,3,27,5,1,1,0,1,0,1,0
4651,2012,3,30,2,0,0,0,0,0,1,1


In [5]:
data.columns

Index(['JoiningYear', 'PaymentTier', 'Age', 'ExperienceInCurrentDomain',
       'LeaveOrNot', 'Education_Masters', 'Education_PHD', 'City_New Delhi',
       'City_Pune', 'Gender_Male', 'EverBenched_Yes'],
      dtype='object')

In [6]:
targets = data['LeaveOrNot']
inputs = data.drop(['LeaveOrNot'],axis=1)

In [7]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(inputs)

scaled = scaler.transform(inputs)

inputs_scaled = pd.DataFrame(scaled, columns=inputs.columns)

inputs_scaled

,JoiningYear,PaymentTier,Age,ExperienceInCurrentDomain,Education_Masters,Education_PHD,City_New Delhi,City_Pune,Gender_Male,EverBenched_Yes
0,1.039638,0.537503,0.954645,-1.864901,-0.480575,-0.200022,-0.575282,-0.612041,0.821551,-0.338365
1,-1.107233,-3.025177,-0.288732,0.060554,-0.480575,-0.200022,-0.575282,1.633878,-1.217210,-0.338365
2,-0.570515,0.537503,1.783563,-0.581264,-0.480575,-0.200022,1.738277,-0.612041,-1.217210,-0.338365
3,0.502921,0.537503,-0.495961,1.344191,2.080840,-0.200022,-0.575282,-0.612041,0.821551,-0.338365
4,1.039638,0.537503,-1.117650,-0.581264,2.080840,-0.200022,-0.575282,1.633878,0.821551,2.955387
...,...,...,...,...,...,...,...,...,...,...
4648,-1.107233,0.537503,-0.703191,0.702373,-0.480575,-0.200022,-0.575282,-0.612041,-1.217210,-0.338365
4649,-1.107233,-1.243837,1.576334,-0.581264,2.080840,-0.200022,-0.575282,1.633878,0.821551,-0.338365
4650,1.576356,0.537503,-0.495961,1.344191,2.080840,-0.200022,1.738277,-0.612041,0.821551,-0.338365
4651,-1.643951,0.537503,0.125727,-0.581264,-0.480575,-0.200022,-0.575282,-0.612041,0.821551,2.955387


In [8]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(inputs_scaled, targets, test_size=0.2, random_state=42)

# 🏗️ Modeling with Artificial Neural Network (ANN)
🧱 Model Architecture:
-Built a deep learning model using Keras Sequential API:

- Input layer matching feature dimensions

- Hidden layers with ReLU activation

- Dropout layers to prevent overfitting

- Output layer with Sigmoid activation for binary classification

🎛️ Hyperparameter Tuning with Optuna:
- Used Optuna to find optimal values for:

- Number of hidden layers and neurons

- Activation functions

- Learning rate

- Batch size and number of epochs

🧪 Model Training & Evaluation:
-Trained the optimized ANN using best hyperparameters. Evaluated performance using:

- Accuracy

- Precision, Recall, F1-Score

- ROC-AUC

In [10]:
ann = Sequential()

In [12]:
def create_model(trial):
    # Building artificial neural network
    model = Sequential()

     # we add 2 hidden layers and 1 output layer
    model.add(Dense(units=trial.suggest_int('units_layer1', 16, 256), activation='relu'))
    model.add(Dense(units=trial.suggest_int('units_layer2', 16, 256), activation='relu'))

    model.add(Dense(units=1, activation='sigmoid'))

    # Suggest hyperparameters for the optimizer
    optimizer_name = trial.suggest_categorical('optimizer', ['adam', 'sgd', 'rmsprop', 'adagrad'])
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-2)
    
    if optimizer_name == 'adam':
        optimizer = Adam(learning_rate=learning_rate)
    elif optimizer_name == 'sgd':
        optimizer = SGD(learning_rate=learning_rate)
    elif optimizer_name == 'rmsprop':
        optimizer = RMSprop(learning_rate=learning_rate)
    elif optimizer_name == 'adagrad':
        optimizer = Adagrad(learning_rate=learning_rate)
    
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['AUC'])
    
    return model

In [13]:
def optimal(trial):
    
    # Suggest the number of epochs and batch size
    epochs = trial.suggest_int('epochs', 10, 50)
    batch_size = trial.suggest_int('batch_size', 16, 64)
    
    model = create_model(trial)
    
    history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size)
    
    y_pred = model.predict(X_test)
    auc = roc_auc_score(y_test, y_pred)
    
    return auc

study = optuna.create_study(direction='maximize')
study.optimize(optimal, n_trials=10)

print(f"Best trial: {study.best_trial.value}")
print(f"Best hyperparameters: {study.best_trial.params}")

[I 2025-05-14 23:35:58,149] A new study created in memory with name: no-name-643d24b9-1ce7-48b8-a9ec-5e26ed634ee0


Epoch 1/33


C:\Users\NIcat\AppData\Local\Temp\ipykernel_14828\312609459.py:13: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-2)


71/71 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - AUC: 0.5093 - loss: 0.6569
Epoch 2/33
71/71 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - AUC: 0.6497 - loss: 0.6305
Epoch 3/33
71/71 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - AUC: 0.7020 - loss: 0.6113
Epoch 4/33
71/71 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - AUC: 0.7459 - loss: 0.5921
Epoch 5/33
71/71 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - AUC: 0.7558 - loss: 0.5825
Epoch 6/33
71/71 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - AUC: 0.7572 - loss: 0.5673
Epoch 7/33
71/71 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - AUC: 0.7681 - loss: 0.5554
Epoch 8/33
71/71 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - AUC: 0.7647 - loss: 0.5592
Epoch 9/33
71/71 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - AUC: 0.7818 - loss: 0.5490
Epoch 10/33
71/71 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - AUC: 0.7793 - loss: 0.5398
Epoch 11/33
71/71 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - AUC: 0.7777 - loss: 0.5279
Epoch 12/33
71/71 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - AUC: 0.7821 - loss: 0.5318
Epoch 13/33
71/71 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - AUC: 0.

[I 2025-05-14 23:36:06,962] Trial 0 finished with value: 0.8503472754200501 and parameters: {'epochs': 33, 'batch_size': 53, 'units_layer1': 250, 'units_layer2': 207, 'optimizer': 'adam', 'learning_rate': 2.1457423334836293e-05}. Best is trial 0 with value: 0.8503472754200501.


...

Epoch 1/23


C:\Users\NIcat\AppData\Local\Temp\ipykernel_14828\312609459.py:13: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-2)


75/75 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - AUC: 0.6412 - loss: 0.6361
Epoch 2/23
75/75 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - AUC: 0.7468 - loss: 0.5622
Epoch 3/23
75/75 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - AUC: 0.7752 - loss: 0.5280
Epoch 4/23
75/75 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - AUC: 0.7802 - loss: 0.5182
Epoch 5/23
75/75 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - AUC: 0.7741 - loss: 0.5211
Epoch 6/23
75/75 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - AUC: 0.7969 - loss: 0.4977
Epoch 7/23
75/75 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - AUC: 0.8111 - loss: 0.4893
Epoch 8/23
75/75 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - AUC: 0.8253 - loss: 0.4700
Epoch 9/23
75/75 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - AUC: 0.8192 - loss: 0.4699
Epoch 10/23
75/75 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - AUC: 0.8269 - loss: 0.4672
Epoch 11/23
75/75 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - AUC: 0.8349 - loss: 0.4521
Epoch 12/23
75/75 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - AUC: 0.8277 - loss: 0.4609
Epoch 13/23
75/75 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - AUC: 0.

[I 2025-05-14 23:37:16,598] Trial 9 finished with value: 0.8776824472703129 and parameters: {'epochs': 23, 'batch_size': 50, 'units_layer1': 249, 'units_layer2': 145, 'optimizer': 'rmsprop', 'learning_rate': 0.00013439965662162933}. Best is trial 8 with value: 0.8976226954700985.


Best trial: 0.8976226954700985
Best hyperparameters: {'epochs': 42, 'batch_size': 31, 'units_layer1': 27, 'units_layer2': 110, 'optimizer': 'rmsprop', 'learning_rate': 0.0010923807647814207}


In [14]:
best_params = study.best_trial.params

best_params

{'epochs': 42,
 'batch_size': 31,
 'units_layer1': 27,
 'units_layer2': 110,
 'optimizer': 'rmsprop',
 'learning_rate': 0.0010923807647814207}

In [15]:
# Train the final model with the best hyperparameters

best_model = Sequential()
best_model.add(Dense(units=best_params['units_layer1'], activation='relu'))
best_model.add(Dense(units=best_params['units_layer2'], activation='relu'))

best_model.add(Dense(1, activation='sigmoid'))

In [16]:
if best_params['optimizer'] == 'adam':
    best_optimizer = Adam(learning_rate=best_params['learning_rate'])
elif best_params['optimizer'] == 'sgd':
    best_optimizer = SGD(learning_rate=best_params['learning_rate'])
elif best_params['optimizer'] == 'rmsprop':
    best_optimizer = RMSprop(learning_rate=best_params['learning_rate'])
elif best_params['optimizer'] == 'adagrad':
    best_optimizer = Adagrad(learning_rate=best_params['learning_rate'])

In [17]:
best_model.compile(optimizer=best_optimizer, loss='binary_crossentropy', metrics=['AUC'])

In [18]:
def evaluate(model, X_train, y_train, X_test, y_test):
    
    model.fit(X_train, y_train, epochs=38, batch_size=best_params['batch_size'])
    
    '''Predictions and probabilities for the training set'''
    
    y_train_prob = model.predict(X_train)

    '''Predictions and probabilities for the test set'''
    
    y_test_prob = model.predict(X_test)

    '''Calculate metrics for the training set''' 
    
    roc_train_prob = roc_auc_score(y_train, y_train_prob)
    gini_train_prob = roc_train_prob * 2 - 1
    

    '''Calculate metrics for the test set'''
    
    roc_test_prob = roc_auc_score(y_test, y_test_prob)
    gini_test_prob = roc_test_prob * 2 - 1
    

    results = pd.DataFrame({
        'Dataset': ['Train', 'Test'],
        'Gini': [gini_train_prob * 100, gini_test_prob * 100],
    
    })

    return results

In [19]:
evaluate(best_model, X_train, y_train, X_test, y_test)

Epoch 1/38
121/121 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - AUC: 0.6430 - loss: 0.6188 
Epoch 2/38
121/121 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - AUC: 0.7988 - loss: 0.5078
Epoch 3/38
121/121 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - AUC: 0.7991 - loss: 0.4913
Epoch 4/38
121/121 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - AUC: 0.8333 - loss: 0.4529
Epoch 5/38
121/121 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - AUC: 0.8471 - loss: 0.4322
Epoch 6/38
121/121 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - AUC: 0.8579 - loss: 0.4203
Epoch 7/38
121/121 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - AUC: 0.8494 - loss: 0.4307
Epoch 8/38
121/121 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - AUC: 0.8608 - loss: 0.4088
Epoch 9/38
121/121 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - AUC: 0.8581 - loss: 0.4114
Epoch 10/38
121/121 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - AUC: 0.8541 - loss: 0.4126
Epoch 11/38
121/121 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - AUC: 0.8654 - loss: 0.4005
Epoch 12/38
121/121 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - AUC: 0.8656 - loss: 0.3907
Epoch 13/38
121/121 ━━━━

,Dataset,Gini
0,Train,80.533266
1,Test,79.770185


# 🚀 Deployment 
📥 Input new employee data

- New employee information is entered directly as a dictionary or a DataFrame.

🧼 Convert categorical columns to numeric format

- Categorical variables like Education, Gender, City, and EverBenched are converted using One-Hot Encoding (get_dummies()).

🧩 Align columns with what the model expects

- Check if any columns used during training are missing in the new data; if so, add them with default value 0.

- Make sure the column order matches the training data.

📏 Scale the data using the same scaler

- Use the StandardScaler instance that was fit during training to transform the new data accordingly (without saving/loading from disk).

🧠 Make predictions using the trained ANN model

- Feed the scaled data into the trained ANN model to get predictions (whether the employee will leave or not).

📋 Add the prediction to the results

- Append the prediction (e.g., a leaveornot column with values 0 or 1) to the original data to present the final result.

In [46]:
df = {
    'Education': ['Bachelors', 'PHD', 'Masters', 'PHD', 'Masters'],
    'JoiningYear': [2015, 2016, 2017, 2018, 2019],
    'City': ['Bangalore', 'Pune', 'New Delhi', 'New Delhi', 'Pune'],
    'PaymentTier': [1, 2, 1, 3, 2],
    'Age': [25, 30, 28, 35, 32],
    'Gender': ['Male', 'Female', 'Male', 'Male', 'Female'],
    'EverBenched': ['No', 'Yes', 'No', 'Yes', 'No'],
    'ExperienceInCurrentDomain': [1, 3, 2, 4, 5]
}

df1 = pd.DataFrame(df)
df1


,Education,JoiningYear,City,PaymentTier,Age,Gender,EverBenched,ExperienceInCurrentDomain
0,Bachelors,2015,Bangalore,1,25,Male,No,1
1,PHD,2016,Pune,2,30,Female,Yes,3
2,Masters,2017,New Delhi,1,28,Male,No,2
3,PHD,2018,New Delhi,3,35,Male,Yes,4
4,Masters,2019,Pune,2,32,Female,No,5


In [54]:
df1=pd.get_dummies(df1,drop_first = True,dtype = int )
df1

,JoiningYear,PaymentTier,Age,ExperienceInCurrentDomain,Education_Masters,Education_PHD,City_New Delhi,City_Pune,Gender_Male,EverBenched_Yes
0,2015,1,25,1,0,0,0,0,1,0
1,2016,2,30,3,0,1,0,1,0,1
2,2017,1,28,2,1,0,1,0,1,0
3,2018,3,35,4,0,1,1,0,1,1
4,2019,2,32,5,1,0,0,1,0,0


In [56]:
scaler.fit(df1)

scaled = scaler.transform(df1)

df_test_scaled = pd.DataFrame(scaled, columns=df1.columns)

df_test_scaled

,JoiningYear,PaymentTier,Age,ExperienceInCurrentDomain,Education_Masters,Education_PHD,City_New Delhi,City_Pune,Gender_Male,EverBenched_Yes
0,-1.414214,-1.069045,-1.468051,-1.414214,-0.816497,-0.816497,-0.816497,-0.816497,0.816497,-0.816497
1,-0.707107,0.267261,0.000000,0.000000,-0.816497,1.224745,-0.816497,1.224745,-1.224745,1.224745
2,0.000000,-1.069045,-0.587220,-0.707107,1.224745,-0.816497,1.224745,-0.816497,0.816497,-0.816497
3,0.707107,1.603567,1.468051,0.707107,-0.816497,1.224745,1.224745,-0.816497,0.816497,1.224745
4,1.414214,0.267261,0.587220,1.414214,1.224745,-0.816497,-0.816497,1.224745,-1.224745,-0.816497


In [58]:
df_test_scaled=df_test_scaled[['JoiningYear','PaymentTier','Age','ExperienceInCurrentDomain','Education_Masters','Education_PHD','City_New Delhi','City_Pune','Gender_Male','EverBenched_Yes']]

In [60]:
df1['leaveornot'] = best_model.predict(df_test_scaled)


df1

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step


,JoiningYear,PaymentTier,Age,ExperienceInCurrentDomain,Education_Masters,Education_PHD,City_New Delhi,City_Pune,Gender_Male,EverBenched_Yes,leaveornot
0,2015,1,25,1,0,0,0,0,1,0,0.560165
1,2016,2,30,3,0,1,0,1,0,1,0.934078
2,2017,1,28,2,1,0,1,0,1,0,0.397841
3,2018,3,35,4,0,1,1,0,1,1,0.023787
4,2019,2,32,5,1,0,0,1,0,0,0.984507
